In [1]:
from ipywidgets import Dropdown, interact
import ipywidgets as widgets
import altair as alt
import pandas as pd
import geopandas as gpd
import gpdvega
from pathlib import Path
from IPython.display import clear_output
%run theme.py

alt.renderers.enable('kaggle')
clear_output()

In [2]:
DATOS_PROCESADOS = Path("../../datos/procesados/")

In [3]:
data = pd.read_csv(DATOS_PROCESADOS / "agebs.csv")

In [4]:
shapes = DATOS_PROCESADOS.glob("*.shp")

In [5]:
geodatos = pd.concat((gpd.read_file(file) for file in shapes), ignore_index = True)

In [6]:
geodatos.set_index("CVEGEO", inplace = True)
data.set_index("CVEGEO", inplace = True)

In [7]:
datos = geodatos.join(data, how = 'right',)

In [8]:
_orden = ['(70, 100]', '(50, 70]', '(34, 50]', '(18, 34]', '[ 0, 18]', 'Una vivienda particular habitada', 'Sin viviendas particulares habitadas', 'No disponible']
_colores = ["#dc0d12","#e9686b","#fbe1e1","#dff4f9","#81d1e6","#dc0d7a","#e46800","#a20c4b"]

_escala = alt.Scale(domain = _orden, range = _colores)

In [9]:
def crea_visualizacion(datos):
    datos.dropna(inplace = True)
    _estado = datos['Entidad Federativa'].unique()[0]
    
    if len(datos['Municipio']) > 1:
        _municipio = "Estado"
    else:
        _municipio = datos['Municipio'].unique()[0]
        
    if _municipio == "Estado":
        _titulo = f"{_estado}"
    else:
        _titulo = f"{_estado} - {_municipio}"
    # base
    mapa_base = alt.Chart(datos).mark_geoshape(
        stroke = '#C7C7C7',
        strokeWidth = 0.1,
    ).encode().properties(
        width = 600,
        height = 600,
        title = _titulo
    )

    # colores
    mapa_resultados = alt.Chart(datos).mark_geoshape(
        fill = 'lightgray',
        stroke = '#C7C7C7',
    ).encode(
        color = alt.Color(
            "Rango de pobreza (%):N", 
            sort = _orden, 
            legend = alt.Legend(
                title = "Rango de pobreza (%) por AGEB", 
                values = _orden,
                orient = 'none',
                legendX = 620,
                legendY = 200,
            ),
            scale = _escala,
        ),
        tooltip = ['Rango de pobreza (%):N', 'Rango de pobreza extrema (%):N']
    )
    return mapa_base + mapa_resultados

In [10]:
def filtra_datos(estado, municipio):
    df = datos.copy()
    mask_estado = df['Entidad Federativa'] == estado
    
    # agrega - para todos
    if municipio == "---":
        working_df = df[mask_estado].copy()
    else:
        mask_municipio = df['Municipio'] == municipio
        working_df = df[mask_estado & mask_municipio].copy()
    
    viz = crea_visualizacion(working_df)
    
    return viz

In [11]:
estado_widget = Dropdown(options = list(data['Entidad Federativa'].unique()), description = 'Estado')

municipio_widget = Dropdown(options = ["---"] + list(data[data['Entidad Federativa']== estado_widget.value]['Municipio'].unique()), description = 'Muncipio')

In [12]:
def on_state_changed(change):
    municipio_widget.options = ["---"] + list(data[data['Entidad Federativa'] == estado_widget.value]['Municipio'].unique())

In [13]:
estado_widget.observe(on_state_changed, names = 'value')

# Pobreza Urbana en México

Inspirado por los mapas de [@moaimx](https://twitter.com/moaimx) ([aquí el hilo de twitter](https://twitter.com/moaimx/status/1141534141487468549)) y la más reciente adición al ecosistema Jupyter: [Voilà](https://github.com/QuantStack/voila) por [@QuantStack](https://twitter.com/QuantStack) decidí intentar hacer algo juntando ambos.

Gracias especiales a [@Coneval](https://twitter.com/Coneval) por los datos.

_aclaración_: Esta aplicación web no esta agregando ningún análisis a los datos y en ciertos municipios puede que no se vea tanto detalle por la naturaleza de su geografía (miren Ensenada en Baja California, por ejemplo).

_aclaración 2_: No se como quitar todo ese texto extra asi que fingamos que no esta ahí jaja

In [14]:
interact(filtra_datos, estado=estado_widget, municipio = municipio_widget);

interactive(children=(Dropdown(description='Estado', options=('Aguascalientes', 'Baja California', 'Baja Calif…